# 01

In [ ]:
!nvidia-smi

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

from keras.datasets.fashion_mnist import load_data
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Conv2DTranspose #, BatchNormalization 
from tensorflow.keras.layers import LeakyReLU

# Dis
from tensorflow.keras.layers import Flatten, Reshape, Dropout

# Gan
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [ ]:
noise_dim = 100  

batch_size = 64 

epochs = 100

img_rows, img_cols, channels = 28, 28, 1

optimizer = Adam(0.0002, 0.5)
np.random.seed(10)

## Load data

In [ ]:
(x_train, y_train), (_, _) = load_data() 

x_train = (x_train.astype(np.float32) - 127.5) / 127.5    

print(x_train.shape)

x_train = np.expand_dims(x_train, axis=-1)
print(x_train.shape)
print('min max:', np.min(x_train), np.max(x_train))

In [ ]:
n_samples = 6000

x_train = x_train[np.where(y_train <= 2)[0]][:n_samples]
x_train.shape

In [ ]:
steps_per_epoch = math.ceil(n_samples / batch_size)
print('number of batches:', steps_per_epoch)

In [ ]:
def visual_multi(i):
    nplots = 16
    fig = plt.figure(figsize=(12, 3)) 
    for j in range(nplots):
        plt.subplot(2, 8, j+1)
        plt.imshow(np.squeeze(x_train[i+j]), cmap=plt.cm.gray_r)
        plt.xticks([]); plt.yticks([])
    plt.show()

visual_multi(0)

In [ ]:
image = x_train[1]
plt.imshow(image.reshape((img_rows, img_cols)), cmap=plt.cm.gray_r)
plt.show()

### Generator

In [ ]:
def create_generator():
    generator = Sequential()
    
    d = 7
    generator.add(Dense(d*d*128, input_dim=noise_dim))
    generator.add(LeakyReLU(0.2))  
    
    generator.add(Reshape((d, d, 128)))
    
    generator.add(Conv2DTranspose(128, (4, 4), strides=2, padding='same'))
    generator.add(LeakyReLU(0.2))

    generator.add(Conv2DTranspose(128/2, (4, 4), strides=2, padding='same'))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Conv2D(channels, (3, 3), padding='same', activation='tanh')) 

    generator.compile(loss='binary_crossentropy', optimizer=optimizer)  
    return generator

### Discrim

In [ ]:
def create_discriminator():
    discriminator = Sequential()
    
    discriminator.add(Conv2D(64, (3, 3), padding='same', input_shape=(img_cols, img_rows, channels)))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Conv2D(128, (3, 3), strides=2, padding='same'))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Flatten())
    discriminator.add(Dropout(0.4))
    discriminator.add(Dense(1, activation='sigmoid')) 
    
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer) 
    return discriminator

## GAN

In [ ]:
discriminator = create_discriminator()
generator = create_generator()

discriminator.trainable = False

gan_input = Input(shape=(noise_dim,))
fake_image = generator(gan_input)

gan_output = discriminator(fake_image)

gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
generator.summary()
discriminator.summary()
gan.summary()

In [ ]:
noise = np.random.normal(0, 1, size=(1, noise_dim))
image = generator.predict(noise)

plt.imshow(image.reshape((img_rows, img_cols)), cmap='gray')
plt.show()

In [ ]:
def show_images(noise, size_fig):
    generated_images = generator.predict(noise)  
    plt.figure(figsize=(size_fig[1], size_fig[0]))
    
    for i, image in enumerate(generated_images):
        plt.subplot(size_fig[0], size_fig[1], i+1)
        if channels == 1:
            plt.imshow(image.reshape((img_rows, img_cols)), cmap=plt.cm.gray_r)        
        else:
            plt.imshow(image.reshape((img_rows, img_cols, channels)))
        plt.axis('off')

    plt.show()

In [ ]:

from tqdm import tqdm
for epoch in range(epochs):
    for batch in tqdm(range(steps_per_epoch)):
        noise = np.random.normal(0, 1, size=(batch_size, noise_dim))
        fake_x = generator.predict(noise)

        real_x = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]
        x = np.concatenate((real_x, fake_x))

        disc_y = np.zeros(2*batch_size)
        disc_y[:batch_size] = 0.9

        d_loss = discriminator.train_on_batch(x, disc_y)

        y_gen = np.ones(batch_size)
        g_loss = gan.train_on_batch(noise, y_gen)

    print(f'Epoch: {epoch+1}/{epochs} \t Discriminator Loss: {d_loss:.3f} \
  Generator Loss: {g_loss:.3f}')

    dsp_img = (3, 5)
    noise = np.random.normal(0, 1, size=(dsp_img[0]*dsp_img[1], noise_dim))
    show_images(noise, dsp_img)

In [ ]:
noise = np.random.normal(0, 1, size=(64, noise_dim))
show_images(noise, (8, 8))